In [ ]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir data
!cp '/content/drive/My Drive/pred-analit/GC-Predictive-Analytics-worksheet - Sheet1 - week3.csv' data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
df = pd.read_csv('data/GC-Predictive-Analytics-worksheet - Sheet1 - week3.csv')
df.drop([9, 10])

,cluster,Gender,Age,Ethnicity,Income,Work hours,Health condition,Education,Motivation,Attitude,Intention,Ownership,1,ER,2,ER.1,3,ER.2,4,ER.3,5,ER.4,6,ER.5,7,ER.6,8,ER.7,9,ER.8,10,ER.9,11,ER.10,12,ER.11,13,ER.12,14,ER.13,15,ER.14,16 Recommendation,Actual ER,17 Recommendation,Actual ER.1,18 Recommendation,Estimated ER
0,0.0,Female,44.0,Caucasian,"150,000-199,999",50+ hrs/week,Good,4-year college degree,0.265623,0.526590,0.507169,0.559363,Phy,0.166667,diet,0.714286,diet,0.428571,diet,0.571429,diet,0.750,diet,0.428571,diet,0.857143,diet,1.0,diet,1.0,diet,0.285714,diet,0.571429,diet,0.500000,Phy,0.0,Phy,0.000000,Phy,0.000000,Phy,0.0,diet,0.0,NaN,NaN
1,1.0,Female,37.0,Asian,"100,000-149,999",16-35 hrs/week,Excellent,2-year college degree,0.386128,0.896973,0.736022,0.924759,diet,0.000000,diet,0.428571,diet,1.000000,diet,0.857143,diet,0.875,diet,0.857143,diet,1.000000,diet,1.0,diet,1.0,diet,0.857143,diet,0.857143,diet,1.000000,diet,1.0,diet,0.857143,diet,0.428571,Phy,0.0,diet,0.0,NaN,NaN
2,2.0,Female,60.0,African American,"0-24,999",50+ hrs/week,Excellent,2-year college degree,0.147511,0.593162,0.574159,0.928628,diet,0.000000,Phy,0.142857,Phy,0.000000,Phy,0.571429,Phy,0.000,Phy,0.000000,Phy,0.000000,Phy,0.0,Phy,0.0,Phy,0.000000,Phy,0.000000,Phy,0.000000,diet,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,Female,62.0,African American,"0-24,999",1-15 hrs/week,Fair,"Some college, but no degree",0.492041,0.586625,0.645084,0.799052,Phy,0.000000,diet,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9.0,Female,62.0,African American,"50,000-99,999",1-15 hrs/week,Good,4-year college degree,0.594119,0.563496,0.659809,0.820454,diet,0.285714,diet,1.000000,Phy,0.470588,diet,0.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.0,Male,59.0,Caucasian,"50,000-99,999",36-50 hrs/week,Fair,4-year college degree,0.760602,0.541951,0.750184,0.664756,phy,0.058824,diet,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,Female,28.0,Asian,"50,000-99,999",50+ hrs/week,Good,4-year college degree,0.653543,0.731372,0.751441,0.976597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,Male,60.0,Asian,"150,000-199,999",50+ hrs/week,Good,Some graduate work,0.736708,0.675436,0.760342,0.974514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2.0,Male,57.0,African American,"25,000-49,000",50+ hrs/week,Excellent,Some graduate work,0.550593,0.765072,0.800639,0.804968,diet,0.833333,Phy,0.857143,Phy,0.857143,Phy,1.000000,Phy,1.000,Phy,0.714286,Phy,1.000000,Phy,1.0,Phy,1.0,Phy,1.000000,Phy,1.000000,Phy,0.833333,Phy,1.0,Phy,1.000000,Phy,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
11,1.0,Male,40.0,HIspanic,"0-24,999",50+ hrs/week,Fair,4-year college degree,0.689156,0.647716,0.738081,0.975890,diet,0.000000,diet,0.000000,diet,0.285714,,0.000000,diet,0.125,diet,0.428571,diet,0.000000,diet,0.0,diet,0.0,diet,0.000000,diet,0.000000,diet,0.000000,diet,0.0,diet,0.000000,diet,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
rec_idxs = [str(i) for i in range(1, 15)]
er_idxs = ['ER']
er_idxs.extend(['ER.{}'.format(i) for i in range(1, 14)])
rec_idxs.append('16 Recommendation')
rec_idxs.append('17 Recommendation')
er_idxs.append('Actual ER')
er_idxs.append('Actual ER.1')

In [ ]:
rrecs = []
eers = []
iids = []
for i in range(len(df)):
  recs = []
  ers = []
  for rec, er in zip(rec_idxs, er_idxs):
      recomendation = df[rec][i]
      try:
        engagement = float(df[er][i])
      except ValueError:
        engagement = 0.0
      if pd.isna(recomendation):
        continue
      elif recomendation == 'Phy':
        recomendation = 0
      else:
        recomendation = 1
      recs.append(recomendation)
      ers.append(engagement)
  rrecs.append(recs)
  eers.append(ers)
  gender = df.Gender[i]
  hours = df['Work hours'][i]
  hmap = {'1-15 hrs/week': 0.2, '16-35 hrs/week': 0.4, '36-50 hrs/week': 0.6, '50+ hrs/week': 1.0}
  if hours in hmap:
    hours = hmap[hours]
  else:
    hours = 0.0
  if gender == 'Female':
    gender = 0
  else:
    gender = 1
  motivation = df.Motivation[i]	
  if pd.isna(motivation):
    motivation = 0.0
  attitude = df.Attitude[i]
  if pd.isna(attitude):
    attitude = 0
  intention = df.Intention[i]
  if pd.isna(intention):
    intention = 0
  ownership = df.Ownership[i]
  if pd.isna(ownership):
    ownership = 0
  iids.append(dict(age = df.Age[i] / 82,
                   gender = gender,  
                   motivation = motivation,	
                   attitude = attitude, 
                   intention = intention,
                   ownership = ownership,
                   hours = hours))

In [ ]:
X = []
y = []
for i in range(len(rrecs)):
  for j in range(len(rrecs[i]) - 2):
    r1 = rrecs[i][j]
    r2 = rrecs[i][j+1]
    # X.append([eers[i][j], r2 - r1, r1, r2, iids[i]['age']])
    X.append([eers[i][j], r1, r2, r2-r1, iids[i]['age'], iids[i]['motivation']]) #iids[i]['attitude']])
    y.append(eers[i][j+1])

# Method evaluation

### regular features

In [ ]:
models = []
models.append(('Linear', LinearRegression()))
models.append(('Ridge', Ridge()))
models.append(('Lasso', Lasso()))
models.append(('Elastic', ElasticNet()))
models.append(('RegTree', RandomForestRegressor()))

# evaluate each model in turn
for name, model in models:
    cv_results = cross_val_score(model, X, y, cv=KFold(n_splits=5, shuffle=True), scoring='r2')
    print("{}: {:.3f} ({})".format(name, cv_results.mean(), cv_results))

Linear: 0.723 ([0.54009265 0.8322708  0.71899855 0.71027378 0.81118049])
Ridge: 0.712 ([0.66934832 0.69375118 0.76498869 0.70388141 0.73005214])
Lasso: -0.037 ([-0.06399453 -0.01051811 -0.01757933 -0.04651322 -0.04737808])
Elastic: -0.020 ([-0.0041722  -0.05011129 -0.00077846 -0.04384445 -0.00101781])
RegTree: 0.652 ([0.83678163 0.47297734 0.82569614 0.60647062 0.51732594])


### polynomial features

In [ ]:
poly = PolynomialFeatures(2)
X_ = poly.fit_transform(X)

In [ ]:
models = []
models.append(('Linear', LinearRegression()))
models.append(('Ridge', Ridge()))
models.append(('Lasso', Lasso()))
models.append(('Elastic', ElasticNet()))
models.append(('RegTree', RandomForestRegressor()))

# evaluate each model in turn
for name, model in models:
    cv_results = cross_val_score(model, X_, y, cv=KFold(n_splits=5, shuffle=True), scoring='r2')
    print("{}: {:.3f} ({})".format(name, cv_results.mean(), cv_results))

Linear: 0.748 ([0.726595   0.78119046 0.85172952 0.61998946 0.76014564])
Ridge: 0.667 ([0.78951099 0.80134059 0.60680914 0.47605532 0.66302508])
Lasso: -0.072 ([-0.14482786 -0.0081209  -0.00584067 -0.19729208 -0.00574231])
Elastic: -0.043 ([-0.00111714 -0.16825798 -0.00901059 -0.01928101 -0.01786214])
RegTree: 0.679 ([0.72375808 0.81229985 0.6974162  0.89539155 0.26569682])


# Weekly prediction

In [ ]:
poly = PolynomialFeatures(2)
X_ = poly.fit_transform(X)
model = RandomForestRegressor()
model.fit(X_, y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
for i in [0, 1, 2, 3, 4, 5, 8]:
    r1 = rrecs[i][-1]
    diet_er = model.predict(poly.transform(np.array([eers[i][-1], r1, 1 - r1, 1, iids[i]['age'], iids[i]['motivation']]).reshape(1, -1)))
    phy_er = model.predict(poly.transform(np.array([eers[i][-1], r1, 0 - r1, 0, iids[i]['age'], iids[i]['motivation']]).reshape(1, -1)))
    if diet_er > phy_er:
      print('id: {} recomendation: {} er: {}'.format(i, 'diet', diet_er[0]))
    else:
      print('id: {} recomendation: {} er: {}'.format(i, 'Phy', phy_er[0]))

id: 0 recomendation: diet er: 0.10240440116478787
id: 1 recomendation: Phy er: 0.17285714287099999
id: 2 recomendation: Phy er: 0.5415176070410133
id: 3 recomendation: diet er: 0.12953201957633229
id: 4 recomendation: Phy er: 0.7932996723762981
id: 5 recomendation: Phy er: 0.5359240821297835
id: 8 recomendation: Phy er: 0.9355532509788234


In [ ]:
sum_diet = 0
sum_phy = 0
diet_n = 0
phy_n = 0
for ers_, rec_ in zip(eers, rrecs):
  for i in range(len(ers_)):
    if rec_[i] == 1:
      sum_diet += ers_[i]
      diet_n += 1
    else:
      sum_phy += ers_[i]
      phy_n += 1

diet = sum_diet / diet_n
phy = sum_phy / phy_n

In [ ]:
diet

0.31585912845835434

In [ ]:
phy

0.2400511928913793